# Model Export, Versioning, and Reproducibility
In production systems, we must save the model, version it, reload it, validate it, prepare it for deployment, and ensure reproducibility.

We'll:
- Train a small image classifier
- Save it using multiple formats
- Implement simple versioning
- Reload and verify predictions
- Export a clean inference-ready artifact

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import json
from datetime import datetime

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.9.1


In [2]:
# Load Dataset (Small Subset for Speed)

IMG_SIZE = 128
BATCH_SIZE = 32

(ds_train, ds_val), ds_info = tfds.load(
    "tf_flowers",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

NUM_CLASSES = ds_info.features["label"].num_classes
CLASS_NAMES = ds_info.features["label"].names


In [3]:
# Preprocessing Pipeline

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = (
    ds_train
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

ds_val = (
    ds_val
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)


In [4]:
# Define Model

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [5]:
# Train Briefly

history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=3
)


Epoch 1/3
92/92 [==============================] - 11s 112ms/step - loss: 1.4171 - accuracy: 0.4074 - val_loss: 1.1354 - val_accuracy: 0.5354
Epoch 2/3
92/92 [==============================] - 10s 106ms/step - loss: 0.9977 - accuracy: 0.6069 - val_loss: 0.9861 - val_accuracy: 0.6253
Epoch 3/3
92/92 [==============================] - 10s 104ms/step - loss: 0.7439 - accuracy: 0.7285 - val_loss: 0.9684 - val_accuracy: 0.6526


In [6]:
# Model Versioning
# In production, we never overwrite models, we version them.

# Create Version Directory

base_dir = "model_registry"
os.makedirs(base_dir, exist_ok=True)

version = datetime.now().strftime("v_%Y%m%d_%H%M%S")
model_dir = os.path.join(base_dir, version)

os.makedirs(model_dir)

print("Model version:", version)

# Save Full Model (SavedModel Format) - saves architecture, weights, optimizer state, computation graph

model.save(model_dir)

print("Model saved to:", model_dir)


Model version: v_20260215_185521


INFO:tensorflow:Assets written to: model_registry\v_20260215_185521\assets


INFO:tensorflow:Assets written to: model_registry\v_20260215_185521\assets


Model saved to: model_registry\v_20260215_185521


In [7]:
# Save Metadata (Critical in Production)

metadata = {
    "version": version,
    "img_size": IMG_SIZE,
    "num_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "framework": "TensorFlow",
    "tf_version": tf.__version__,
    "date_saved": str(datetime.now())
}

with open(os.path.join(model_dir, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=4)

print("Metadata saved.")


Metadata saved.


In [8]:
# Reload and Validate

loaded_model = tf.keras.models.load_model(model_dir)

sample_batch = next(iter(ds_val))
images, labels = sample_batch

original_preds = model.predict(images)
loaded_preds = loaded_model.predict(images)

difference = np.mean(np.abs(original_preds - loaded_preds))

print("Mean prediction difference:", difference)


1/1 [==============================] - 0s 115ms/step
Mean prediction difference: 0.0


In [9]:
# Export Weights Only (Optional Alternative)

weights_path = os.path.join(model_dir, "weights_only.h5")
model.save_weights(weights_path)

print("Weights saved to:", weights_path)


Weights saved to: model_registry\v_20260215_185521\weights_only.h5


In [10]:
# Create Inference-Ready Wrapper
# In production, we often separate the training and inference pipelines

# Define Inference Function

def predict_image(model, image_array):
    image = tf.image.resize(image_array, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, axis=0)

    preds = model.predict(image)
    predicted_class = CLASS_NAMES[np.argmax(preds)]

    return predicted_class

# Test Inference

test_image = images[0]
prediction = predict_image(loaded_model, test_image)

print("Predicted class:", prediction)


1/1 [==============================] - 0s 75ms/step
Predicted class: dandelion


We:
- Trained a vision model
- Created a versioned model registry
- Saved model artifacts
- Saved metadata
- Reloaded the model
- Verified prediction consistency
- Built an inference wrapper

In real systems:
- Models must be reproducible
- Models must be versioned
- Models must be traceable
- Models must be deployable independently of training